In [13]:
import RobotControl_func_ros1
import rospy

In [14]:
rospy.init_node('my_node_name')

robot = RobotControl_func_ros1.RobotControl_Func()

In [15]:
pos_home = [496.8341605883501, -98.10461371288869, 434.80804443359375, -179.68991088867188, -0.650927722454071, 44.84828567504883]

In [16]:
[x,y,z,u,v,w] = robot.get_TMPos()
print([x,y,z,u,v,w])

[538.6810169352628, 546.6707505004009, 349.7906188964844, -176.4381866455078, 3.850838899612427, -0.2126985639333725]


In [17]:
#obot.set_TMPos([x,y,z,u,v,w])
robot.set_TMPos(pos_home)

[0.28194436645507814, -0.4206852416992188, 0.4348080444335938, -3.136180577622534, -0.011360831949331364, 0.7827502489046096]
[INFO] [1702957110.826312]: point 1 (Tag 1) is reached


In [1]:
import gripper

In [2]:
g = gripper.Gripper(0)
g.gripper_reset()


Response: b'091003e800030130'
Activate gripper:
Clear: 09 10 03 E8 00 03 06 00 00 00 00 00 00 73 30
Response: b'091003e800030130'
Set: 09 10 03 E8 00 03 06 01 00 00 00 00 00 72 E1
b'\x08h\x10\x0fhP\x10\xe6'
b'\t\x03\x021\x00L\x15'
Activate Complete


#### mediapipe gesture control

In [7]:
import cv2
import mediapipe as mp
import math
import time

pTime = 0
cTime = 0

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

detecting = True

# 根據兩點的座標，計算角度
def vector_2d_angle(v1, v2):
    v1_x = v1[0]
    v1_y = v1[1]
    v2_x = v2[0]
    v2_y = v2[1]
    try:
        angle_= math.degrees(math.acos((v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 180
    return angle_

# 根據傳入的 21 個節點座標，得到該手指的角度
def hand_angle(hand_):
    angle_list = []
    # thumb 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[2][0])),(int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0])- int(hand_[4][0])),(int(hand_[3][1])- int(hand_[4][1])))
        )
    angle_list.append(angle_)
    # index 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),(int(hand_[0][1])- int(hand_[6][1]))),
        ((int(hand_[7][0])- int(hand_[8][0])),(int(hand_[7][1])- int(hand_[8][1])))
        )
    angle_list.append(angle_)
    # middle 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[10][0])),(int(hand_[0][1])- int(hand_[10][1]))),
        ((int(hand_[11][0])- int(hand_[12][0])),(int(hand_[11][1])- int(hand_[12][1])))
        )
    angle_list.append(angle_)
    # ring 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[14][0])),(int(hand_[0][1])- int(hand_[14][1]))),
        ((int(hand_[15][0])- int(hand_[16][0])),(int(hand_[15][1])- int(hand_[16][1])))
        )
    angle_list.append(angle_)
    # pink 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[18][0])),(int(hand_[0][1])- int(hand_[18][1]))),
        ((int(hand_[19][0])- int(hand_[20][0])),(int(hand_[19][1])- int(hand_[20][1])))
        )
    angle_list.append(angle_)
    return angle_list

# 根據手指角度的串列內容，返回對應的手勢名稱
def hand_pos(finger_angle):
    f1 = finger_angle[0]   # 大拇指角度
    f2 = finger_angle[1]   # 食指角度
    f3 = finger_angle[2]   # 中指角度
    f4 = finger_angle[3]   # 無名指角度
    f5 = finger_angle[4]   # 小拇指角度

    # 小於 50 表示手指伸直，大於等於 50 表示手指捲縮
    if f1<50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        print("good!")
        print(detecting)
        #time.sleep(1.5)
        return 'good'
    elif f1>=50 and f2>=50 and f3<50 and f4>=50 and f5>=50:
        print('damn')
        return 'no!!!'
    elif f1<50 and f2<50 and f3>=50 and f4>=50 and f5<50:
        print('fuck!')
        return 'ROCK!'
    elif f1>=50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        return '0'
    elif f1>=50 and f2>=50 and f3>=50 and f4>=50 and f5<50:
        return 'pink'
    elif f1>=50 and f2<50 and f3>=50 and f4>=50 and f5>=50: #1
        print("grasp!")
        #detecting =  False
        #g.gripper_reset()
        return 'grasp'
    elif f1>=50 and f2<50 and f3<50 and f4>=50 and f5>=50: #2
        print("release!")
        return 'release'
    elif f1>=50 and f2>=50 and f3<50 and f4<50 and f5<50:
        return 'ok'
    elif f1<50 and f2>=50 and f3<50 and f4<50 and f5<50:
        return 'ok'
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5>50:
        return '3'
    elif f1>=50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '4'
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5<50:
        return '5'
    elif f1<50 and f2>=50 and f3>=50 and f4>=50 and f5<50:
        return '6'
    elif f1<50 and f2<50 and f3>=50 and f4>=50 and f5>=50:
        return '7'
    elif f1<50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return '8'
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5>=50:
        return '9'
    else:
        return ''

cap = cv2.VideoCapture(0)            # 讀取攝影機
fontFace = cv2.FONT_HERSHEY_SIMPLEX  # 印出文字的字型
lineType = cv2.LINE_AA               # 印出文字的邊框

# mediapipe 啟用偵測手掌
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()
    w, h = 1080, 620                                  # 影像尺寸
    while detecting:
        ret, img = cap.read()
        img = cv2.resize(img, (w,h))                 # 縮小尺寸，加快處理效率
        if not ret:
            print("Cannot receive frame")
            break

        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # 轉換成 RGB 色彩
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        cv2.putText(img, f"FPS : {int(fps)}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
                  
        results = hands.process(img2)                # 偵測手勢
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                finger_points = []                   # 記錄手指節點座標的串列
                for i in hand_landmarks.landmark:
                    # 將 21 個節點換算成座標，記錄到 finger_points
                    x = i.x*w
                    y = i.y*h
                    finger_points.append((x,y))

                    # 編號 21 個手部節點
                # for i, lm in enumerate(hand_landmarks.landmark):
                #     if i==9: # 中指根部
                #         print(lm.x, " ", lm.y) #lm.x, lm,y = [0,1]
        
                if finger_points:
                    finger_angle = hand_angle(finger_points) # 計算手指角度，回傳長度為 5 的串列
                    #print(finger_angle)                     # 印出角度 ( 有需要就開啟註解 )
                    text = hand_pos(finger_angle)            # 取得手勢所回傳的內容
                    cv2.putText(img, text, (30,120), fontFace, 5, (255,255,255), 10, lineType) # 印出文字
                    if(text == 1):
                        time.sleep(3)

        cv2.imshow('result', img)
        if cv2.waitKey(5) == ord('q'):
            break
        if cv2.waitKey(5) == 27:
            break
cap.release()
cv2.destroyAllWindows()


# 
# detecting = 1
# until progress finished
# back to detect
# 1:grasp 2:release 
# point9 coordinate -> gripper movement

I0000 00:00:1702956797.931687  164939 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702956797.934338  165039 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) Graphics (ADL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


good!
True
good!
True
good!
True
good!
True
grasp!
[0.8017123731455432, 0.02786816974204237, 0.39366339111328125, -3.0743265307765713, -0.001811583212347239, 18.84955592153876]
service [/tm_driver/ask_sta] responded with an error: b''
grasp!
[0.8020715662902294, 0.022201656614983106, 0.39366339111328125, -3.0743265307765713, -0.001811583212347239, 18.84955592153876]
service [/tm_driver/set_positions] responded with an error: b''
good!
True
grasp!
[0.796318871287335, 0.01427981841963549, 0.39366339111328125, -3.0743265307765713, -0.001811583212347239, 18.84955592153876]
service [/tm_driver/set_positions] responded with an error: b''
good!
True
grasp!
[0.7813970737570555, -0.009369532708415534, 0.39366339111328125, -3.0743265307765713, -0.001811583212347239, 18.84955592153876]
service [/tm_driver/set_positions] responded with an error: b''
grasp!
[0.786633396530175, -0.003360986947509673, 0.39366339111328125, -3.0743265307765713, -0.001811583212347239, 18.84955592153876]
service [/tm_dri

Open gripper
Response: b'091003e800030130'
